# PROJ631 – Projet algorithmique
## Compte rendu Emma ORSET
## Compression de données par codage de Huffman

In [552]:
import os

## Etape 1 : Détermination de l’alphabet et des fréquences de caractères

Je récupère le fichier qui contient mon texte. Je l'ouvre puis je le lis. 
Cela créer une chaine de caractère.

In [553]:
#file_name = "extraitalice"
#file_name = "alice"
#file_name = "textesimple"
#file_name = "bonjour"
file_name = "texteperso"
with open(file_name +"/"+file_name + ".txt","r", encoding="UTF-8") as f:
    text = f.read()

print(text)

Je crée ce texte pour   mettre les caractères que je souhaite afin de voir si ça marche. 
Des chiffres et autres caractères : 2, "34", (28), |, [], °, @, =, £, *, µ.


Je crée une liste qui contient chaque caractère de mon texte et je la trie selon l'ordre de la table ASCII.

In [554]:
list_sorted_label = []

for i in text:
    
    list_sorted_label.append(i)

list_sorted_label.sort()

print(list_sorted_label)

['\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '"', '"', '(', ')', '*', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', '.', '.', '2', '2', '3', '4', '8', ':', '=', '@', 'D', 'J', '[', ']', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'd', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'f', 'f', 'f', 'h', 'h', 'h', 'i', 'i', 'i', 'i', 'i', 'j', 'l', 'm', 'm', 'n', 'o', 'o', 'o', 'p', 'q', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 's', 's', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 't', 't', 'u', 'u', 'u', 'u', 'v', 'x', '|', '£', '°', 'µ', 'ç', 'è', 'è', 'é']


Je créer deux listes :
- list_label contient une ocurrence de chaque élement de la liste triée ci-dessus
- list_freq contient la fréquence de ces éléments

Cela crée mon alphabet

In [555]:
list_label = []
list_freq = []

for i in list_sorted_label:
    if i not in list_label:
        list_label.append(i)
        list_freq.append(list_sorted_label.count(i))

print(list_label)
print(list_freq)

['\n', ' ', '"', '(', ')', '*', ',', '.', '2', '3', '4', '8', ':', '=', '@', 'D', 'J', '[', ']', 'a', 'c', 'd', 'e', 'f', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', '|', '£', '°', 'µ', 'ç', 'è', 'é']
[1, 35, 2, 1, 1, 1, 10, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 8, 1, 19, 3, 3, 5, 1, 1, 2, 1, 3, 1, 1, 11, 8, 9, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1]


Afin de trier mes deux listes selon la liste des fréquences, j'utilise des tuples.

In [556]:
list_sorted_freq = zip(list_freq,list_label)
list_sorted_freq = sorted(list_sorted_freq)
list_sorted_freq = zip(*list_sorted_freq)

list_freq, list_label = [[*tuple] for tuple in list_sorted_freq]

print(list_freq)
print(list_label)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 4, 5, 8, 8, 9, 9, 10, 11, 19, 35]
['\n', '(', ')', '*', '3', '4', '8', ':', '=', '@', 'D', 'J', '[', ']', 'd', 'j', 'l', 'n', 'p', 'q', 'v', 'x', '|', '£', '°', 'µ', 'ç', 'é', '"', '.', '2', 'm', 'è', 'f', 'h', 'o', 'u', 'i', 'c', 's', 'a', 't', ',', 'r', 'e', ' ']


Il est demandé d'afficher le nombre de caractères que contient notre nouvel alphabet.
J'affiche ensuite chaque caractère et sa fréquence associée comme demandé dans le sujet.

In [557]:
length_dict = len(list_label)
text_freq = str(length_dict) + "\n\n"

for i in range (length_dict):
    if(list_label[i]=="\n"):
        text_freq = text_freq + "\\n" + " " + str(list_freq[i]) + "\n"
    else:
        text_freq = text_freq + list_label[i] + " " + str(list_freq[i]) + "\n"

print(text_freq)

46

\n 1
( 1
) 1
* 1
3 1
4 1
8 1
: 1
= 1
@ 1
D 1
J 1
[ 1
] 1
d 1
j 1
l 1
n 1
p 1
q 1
v 1
x 1
| 1
£ 1
° 1
µ 1
ç 1
é 1
" 2
. 2
2 2
m 2
è 2
f 3
h 3
o 3
u 4
i 5
c 8
s 8
a 9
t 9
, 10
r 11
e 19
  35



J'écris mon dictionnaire dans un fichier en écrasant l'ancien contenu si le fichier existait déjà.

In [558]:
new_file = open(file_name +"/"+file_name + "_freq.txt", "w" ,encoding="UTF-8")
new_file.write(text_freq)
new_file.close()

Maintenant que j'ai mon dictionnaire, je peux commencer à compresser mon texte grâce à la technique de l'arbre de Huffman.

## Etape 2 : Construction de l’arbre 

Dans un premier temps, je crée une classe Node.
Je passe en paramètre :
- label = le caractère (a)
- freq = la fréquence de mon charactère dans le texte (4)
- le noeud enfant de gauche et le noeud enfant de droite si ces derniers existent, sinon ça veut dire que c'est une feuille.

In [559]:
class node:
    def __init__(self, label, freq, left_child=None, right_child=None):
         # Label de la lettre : "a"
        self.label = label
 
        # Fréquence de la lettre : 4
        self.freq = freq
 
        # L'enfant gauche(0) de la lettre en question : de type node
        self.left_child = left_child
 
        # L'enfant droit(1) de la lettre en question : de type node
        self.right_child = right_child
 
        # Direction : 0 ou 1
        self.direction = ''

    def __str__(self):
        return str(self.label) + " " + str(self.freq) + " " + str(self.left_child) + " " + str(self.right_child)

Pour créer l'algorithme, j'ai d'abord créé les feuilles.
Chaque caractère et sa fréquence sont donc une feuille que j'ajoute à une liste.

In [560]:
list_node = []

for i in range(len(list_label)):
    list_node.append(node(list_label[i], list_freq[i]))

for i in range(len(list_node)):
    print(list_node[i])


 1 None None
( 1 None None
) 1 None None
* 1 None None
3 1 None None
4 1 None None
8 1 None None
: 1 None None
= 1 None None
@ 1 None None
D 1 None None
J 1 None None
[ 1 None None
] 1 None None
d 1 None None
j 1 None None
l 1 None None
n 1 None None
p 1 None None
q 1 None None
v 1 None None
x 1 None None
| 1 None None
£ 1 None None
° 1 None None
µ 1 None None
ç 1 None None
é 1 None None
" 2 None None
. 2 None None
2 2 None None
m 2 None None
è 2 None None
f 3 None None
h 3 None None
o 3 None None
u 4 None None
i 5 None None
c 8 None None
s 8 None None
a 9 None None
t 9 None None
, 10 None None
r 11 None None
e 19 None None
  35 None None


L’algorithme  repose sur une structure d’arbre binaire où tous les nœuds internes ont exactement deux successeurs. 
Les feuilles sont étiquetées avec les caractères de l’alphabet.
Les branches sont étiquetées par 0 (fils gauche) et 1 (fils droit). 
Les chemins depuis la racine jusqu’aux feuilles constituent les codes des caractères.  

Je fais une boucle qui s'execute tant qu'on n'est pas à la racine de l'arbre.

In [561]:
while len(list_node) > 1:

    list_node = sorted(list_node, key=lambda x: x.freq)
 
    # Affectation des deux noeuds les plus petits
    node_left = list_node[0]
    node_right = list_node[1]
 
    # Affectation d'une direction à ces deux noeuds
    node_left.direction = 0
    node_right.direction = 1

    # La fréquence du parent devient la somme de la fréquence de ses deux enfants
    freq_parent = node_left.freq + node_right.freq

    # Enfin, on crée le noued parents avec les informations récoltées
    node_parent = node(freq_parent,freq_parent , node_left, node_right)
 
 
    # Suppression des noeuds enfants pour ne garder que le noeud parent
    list_node.remove(node_left)
    list_node.remove(node_right)
    list_node.append(node_parent)

print(list_node[0])

165 165 69 69 34 34 16 16 8 8 4 4 2 2 D 1 None None J 1 None None 2 2 [ 1 None None ] 1 None None 4 4 2 2 d 1 None None j 1 None None 2 2 l 1 None None n 1 None None 8 8 4 4 2 2 p 1 None None q 1 None None 2 2 v 1 None None x 1 None None 4 4 2 2 | 1 None None £ 1 None None 2 2 ° 1 None None µ 1 None None 18 18 a 9 None None t 9 None None   35 None None 96 96 39 39 e 19 None None 20 20 , 10 None None 10 10 i 5 None None 5 5 2 2 ç 1 None None é 1 None None f 3 None None 57 57 25 25 r 11 None None 14 14 6 6 h 3 None None o 3 None None c 8 None None 32 32 16 16 s 8 None None 8 8 u 4 None None 4 4 " 2 None None . 2 None None 16 16 8 8 4 4 2 2 None None m 2 None None 4 4 è 2 None None 2 2 
 1 None None ( 1 None None 8 8 4 4 2 2 ) 1 None None * 1 None None 2 2 3 1 None None 4 1 None None 4 4 2 2 8 1 None None : 1 None None 2 2 = 1 None None @ 1 None None


Comme on peut le voir, on obtient une liste d'un seul élément contenant tous les noeuds.
Pour créer l'arbre, il faut parcourir cet élément récursivement dans le but de créer le code binaire de chaque caractère.

In [562]:
def createListNodes(node, binary, list):
    # Créer le code binaire
    binary = binary + str(node.direction) 

    dict_comp = {}
    
    # Si le noeud contient un enfant gauche ou droit, alors ce n'est pas une feuille et il faut rappeler la fonction
    if (node.left_child):
        createListNodes(node.left_child, binary,list)

    if (node.right_child):
        createListNodes(node.right_child, binary,list)

    # Si le noeud n'a pas d'enfant, c'est une feuille.
    # On peut donc l'ajouter dans le dictionnaire et lui affecter son code binaire
    if(not node.left_child and not node.right_child):
        dict_comp[node.label] = binary
        # Je créer une liste de dictionnaire
        list.append(dict_comp)
        
    return list 

Voici donc à quoi ressemble la liste de dictionnaire après appel de la fonction.

In [563]:
binary = ""
list = []

list_nodes = createListNodes(list_node[0], binary,list )

print(list_nodes)

[{'D': '0000000'}, {'J': '0000001'}, {'[': '0000010'}, {']': '0000011'}, {'d': '0000100'}, {'j': '0000101'}, {'l': '0000110'}, {'n': '0000111'}, {'p': '0001000'}, {'q': '0001001'}, {'v': '0001010'}, {'x': '0001011'}, {'|': '0001100'}, {'£': '0001101'}, {'°': '0001110'}, {'µ': '0001111'}, {'a': '0010'}, {'t': '0011'}, {' ': '01'}, {'e': '100'}, {',': '1010'}, {'i': '10110'}, {'ç': '1011100'}, {'é': '1011101'}, {'f': '101111'}, {'r': '1100'}, {'h': '110100'}, {'o': '110101'}, {'c': '11011'}, {'s': '11100'}, {'u': '111010'}, {'"': '1110110'}, {'.': '1110111'}, {'2': '1111000'}, {'m': '1111001'}, {'è': '1111010'}, {'\n': '11110110'}, {'(': '11110111'}, {')': '11111000'}, {'*': '11111001'}, {'3': '11111010'}, {'4': '11111011'}, {'8': '11111100'}, {':': '11111101'}, {'=': '11111110'}, {'@': '11111111'}]


On peut récupérer la clef, la valeur, ou les les deux de cette manière :

In [564]:
for key in list_nodes[1].keys():
    print (key)

for value in list_nodes[1].values():
    print (value)

for key,value in list_nodes[1].items():
    print(key, value)

J
0000001
J 0000001


## Etape 3 : Codage du texte

Je créer une liste qui contient les caractères de mon texte de base.

In [565]:
list_text = []

for i in text:
    list_text.append(i)
    
print(list_text)

['J', 'e', ' ', 'c', 'r', 'é', 'e', ' ', 'c', 'e', ' ', 't', 'e', 'x', 't', 'e', ' ', 'p', 'o', 'u', 'r', ' ', ' ', ' ', 'm', 'e', 't', 't', 'r', 'e', ' ', 'l', 'e', 's', ' ', 'c', 'a', 'r', 'a', 'c', 't', 'è', 'r', 'e', 's', ' ', 'q', 'u', 'e', ' ', 'j', 'e', ' ', 's', 'o', 'u', 'h', 'a', 'i', 't', 'e', ' ', 'a', 'f', 'i', 'n', ' ', 'd', 'e', ' ', 'v', 'o', 'i', 'r', ' ', 's', 'i', ' ', 'ç', 'a', ' ', 'm', 'a', 'r', 'c', 'h', 'e', '.', ' ', '\n', 'D', 'e', 's', ' ', 'c', 'h', 'i', 'f', 'f', 'r', 'e', 's', ' ', 'e', 't', ' ', 'a', 'u', 't', 'r', 'e', 's', ' ', 'c', 'a', 'r', 'a', 'c', 't', 'è', 'r', 'e', 's', ' ', ':', ' ', '2', ',', ' ', '"', '3', '4', '"', ',', ' ', '(', '2', '8', ')', ',', ' ', '|', ',', ' ', '[', ']', ',', ' ', '°', ',', ' ', '@', ',', ' ', '=', ',', ' ', '£', ',', ' ', '*', ',', ' ', 'µ', '.']


Je parcours la liste de mon texte et je remplace chaque caractère par le code binaire qui correspond en parcourant ma liste de dictionnaire.

In [566]:
for i in range(len(list_text)):
    for j in list_nodes:
        for cle,valeur in j.items():
            if (cle == list_text[i]):
                list_text[i] = valeur
print(list_text)

['0000001', '100', '01', '11011', '1100', '1011101', '100', '01', '11011', '100', '01', '0011', '100', '0001011', '0011', '100', '01', '0001000', '110101', '111010', '1100', '01', '01', '01', '1111001', '100', '0011', '0011', '1100', '100', '01', '0000110', '100', '11100', '01', '11011', '0010', '1100', '0010', '11011', '0011', '1111010', '1100', '100', '11100', '01', '0001001', '111010', '100', '01', '0000101', '100', '01', '11100', '110101', '111010', '110100', '0010', '10110', '0011', '100', '01', '0010', '101111', '10110', '0000111', '01', '0000100', '100', '01', '0001010', '110101', '10110', '1100', '01', '11100', '10110', '01', '1011100', '0010', '01', '1111001', '0010', '1100', '11011', '110100', '100', '1110111', '01', '11110110', '0000000', '100', '11100', '01', '11011', '110100', '10110', '101111', '101111', '1100', '100', '11100', '01', '100', '0011', '01', '0010', '111010', '0011', '1100', '100', '11100', '01', '11011', '0010', '1100', '0010', '11011', '0011', '1111010', '1

Je concatène les éléments de ma liste pour obtenir une chaine de caractère.

In [567]:
text_binary = ""
for i in list_text:
    text_binary = text_binary + i

print(text_binary)

0000001100011101111001011101100011101110001001110000010110011100010001000110101111010110001010111110011000011001111001000100001101001110001110110010110000101101100111111010110010011100010001001111010100010000101100011110011010111101011010000101011000111000100101011111011000001110100001001000100010101101011011011000111100101100110111000010011111001001011001101111010010011101110111110110000000010011100011101111010010110101111101111110010011100011000011010010111010001111001001110001110110010110000101101100111111010110010011100011111110101111100010100111101101111101011111011111011010100111110111111100011111100111110001010010001100101001000001000000111010010001110101001111111111010011111111010100100011011010011111100110100100011111110111


Pour que cette chaine soit lu en binaire, je la transforme.
J'ai pris une fonction sur internet qui permet de faire cela.

In [568]:
text_compressed = int(text_binary, base=2).to_bytes((len(text_binary)+7)//8, byteorder='big') 
print(text_compressed)

b'\x00\xc7yv;\x89\xc1g\x11\x1a\xf5\x8a\xf9\x86y\x10\xd3\x8e\xcb\x0bg\xeb\'\x11=D,y\xafZ\x15\x8e%}\x83\xa1"+[c\xcb7\t\xf2Y\xbd\'w\xd8\x04\xe3\xbd-}\xf98\xc3K\xa3\xc9\xc7e\x85\xb3\xf5\x93\x8f\xeb\xe2\x9e\xdf_}\xa9\xf7\xf1\xf9\xf1H\xcaA\x03\xa4u?\xf4\xffR6\x9f\x9aG\xf7'


J'écris ce que j'obtiens dans un fichier.
Le "wb" permet de spécifier qu'on écrit en binaire.

In [569]:
new_file = open(file_name +"/"+file_name + "_comp.bin", "wb")
new_file.write(text_compressed)
new_file.close()

## Etape 4 : Détermination du taux de compression

Tout d'abord, je récupère les tailles de mon ancien fichier texte et mon nouveau fichier binaire.
Je calcul le taux de compression selon la formule qui m'est donnée.

In [570]:
text_compressed_size = os.path.getsize(file_name +"/"+file_name + "_comp.bin") 
print('Taille de '+ file_name + "_comp.bin" +' :', text_compressed_size, 'bits')

text_origin_size = os.path.getsize(file_name +"/"+file_name + '.txt') 
print('Taille de '+ file_name + ".txt" +' :', text_origin_size, 'bits')

amount = 1 - text_compressed_size/text_origin_size
print("Le taux est de :" ,round(amount,3))

Taille de texteperso_comp.bin : 93 bits
Taille de texteperso.txt : 173 bits
Le taux est de : 0.462


## Etape 5 : Détermination du nombre moyen de bits de stockage d'un caractère du texte compressé

Je fais une simple moyenne. 
Je prends chaque code binaire et je regarde le nombre de caractère qu'il y a (ex : 001 = 3; 1110 = 4).
J'additionne tous les chiffres et je divise cela par la longueur de ma list_text.

In [571]:
dividend = 0
diviseur = len(list_text)
for i in range(diviseur):
    dividend += len(list_text[i])

average = dividend/diviseur

print(average)

4.496969696969697
